# Simulations of Yeast

This notebook is based on:

https://github.com/BenjaSanchez/notebooks/blob/8895b5293dadc9fe3d1c785d28523bcf8fc8c65f/e_coli_simulations/notebook.ipynb

And is used to transform the model bla of to comply with the standards required for upload on ddecaf.


Jonas Sindlinger, 2020-02-07

## 1. Loading models

* ec-yeast model: https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml


In [115]:
import cobra
import os
import wget
import pandas as pd
import ssl
import math


# Enzyme-constrained model:
ssl._create_default_https_context = ssl._create_unverified_context
wget.download("https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml", "ecYeastGEM.xml", bar=False)
ecModel = cobra.io.read_sbml_model("ecYeastGEM.xml")
os.remove("ecYeastGEM.xml")
ecModel.summary()

In [116]:
# loading of translation files for mets and reactions 
wget.download("https://raw.githubusercontent.com/SysBioChalmers/yeast-GEM/3b83df2a442d7b874a4f1d0e19e7eeeca6142f1d/ComplementaryData/modelCuration/globalMetDictionary.csv", "globalMetDictionary.csv", bar=False)
pd_metId2name = pd.read_csv("globalMetDictionary.csv", header=None)
pd_metId2name.columns = ["id", "BiGG"]
os.remove("globalMetDictionary.csv")

wget.download("https://raw.githubusercontent.com/SysBioChalmers/yeast-GEM/3b83df2a442d7b874a4f1d0e19e7eeeca6142f1d/ComplementaryData/databases/Yeast7.6MNXMetMappingList.tsv", "mnxMetDict.tsv", bar=False)
pd_metId2mnx = pd.read_csv("mnxMetDict.tsv", sep="\t")
pd_metId2mnx.columns = ["id", "mnx"]
os.remove("mnxMetDict.tsv")

wget.download("https://raw.githubusercontent.com/SysBioChalmers/yeast-GEM/3b83df2a442d7b874a4f1d0e19e7eeeca6142f1d/ComplementaryData/modelCuration/globalRxnDictionary.csv", "globalRxnDictionary.csv", bar=False)
pd_rxnId2name = pd.read_csv("globalRxnDictionary.csv", header=None)
pd_rxnId2name.columns = ["id", "BiGG"]
os.remove("globalRxnDictionary.csv")

wget.download("https://raw.githubusercontent.com/SysBioChalmers/yeast-GEM/3b83df2a442d7b874a4f1d0e19e7eeeca6142f1d/ComplementaryData/databases/Yeast7.6MNXRxnMappingList.tsv", "mnxReactionDict.tsv", bar=False)
pd_rxnId2mnx = pd.read_csv("mnxReactionDict.tsv", sep="\t")
pd_rxnId2mnx.columns = ["id", "mnx"]
os.remove("mnxReactionDict.tsv")


# creating dictionary from pandas df
dict_metId2name = pd_metId2name.set_index("id").to_dict()["BiGG"]
dict_rxnId2name = pd_rxnId2name.set_index("id").to_dict()["BiGG"]

dict_metId2mnx = pd_metId2mnx.set_index("id").to_dict()["mnx"]
dict_rxnId2mnx = pd_rxnId2mnx.set_index("id").to_dict()["mnx"]


### 2.1 Set Metabolite names to BiGG format 

In [117]:
%%capture
# translation of ids to BiGG and annotations to mnx
# [setattr(met,'annotation',) for met in ecModel.metabolites if met.id in dict_metId2mnx]
for met in ecModel.metabolites:
    if met.id in dict_metId2mnx:
        met.annotation['metanetx.chemical'] = dict_metId2mnx[met.id]
[setattr(met,'id',dict_metId2name[met.id]) for met in ecModel.metabolites if met.id in dict_metId2name]

### 2.2 Set Reaction names to BiGG format

In [118]:
%%capture
### translation of ids to BiGG and annotations to mnx

list_ids = [rxn.id for rxn in ecModel.reactions]

# substitute annotations 
for rxn in ecModel.reactions:
    if (re.findall(r"r_\d{4}|$",rxn.id)[0] in dict_rxnId2mnx and re.findall(r"r_\d{4}|$",rxn.id)[0] != ''):
           rxn.annotation["metanetx.reaction"] = dict_rxnId2mnx[re.findall(r"r_\d{4}|$",rxn.id)[0]]


# substitute reaction names 
subst_list_ids = [re.sub(r"r_\d{4}", dict_rxnId2name[re.findall(r"r_\d{4}",rxn)[0]], rxn) \
                if (re.findall(r"r_\d{4}|$",rxn)[0] in dict_rxnId2name and re.findall(r"r_\d{4}|$",rxn)[0] != '') \
                else rxn \
                for rxn in list_ids]

# insert reaction names
[setattr(ecModel.reactions[i], 'id', subst_list_ids[i]) for i in range(0,len(ecModel.reactions))]


### 2 Fixing the enzyme-constrained model

cobrapy cannot handle upper bounds = `Inf`, therefore we need to replace them with `1000` (standard in the field):

In [124]:
# necessary
for reaction in ecModel.reactions:
    if math.isinf(reaction.upper_bound):
        reaction.upper_bound = 1000
        
        

ecModel.optimize()

,fluxes,reduced_costs
16BETGLUSYN,0.022002,0.000000
4HBZFm,0.000000,0.000000
AOXSr_copy,0.000000,0.000000
ACOATAm,0.000000,0.000000
AACTOOR,0.000000,-0.041053
...,...,...
prot_Q99190_exchange,0.000000,-0.000000
prot_Q99258_exchange,0.000000,-0.000000
prot_Q99288_exchange,0.000000,-0.000000
prot_Q99312_exchange,0.000000,0.000000


### 3. Fixing metabolite name suffixes

In [125]:
# Metabolite ids: remove the trailing "[comp]" (if any):
for metabolite in ecModel.metabolites:
    trail = "[" + metabolite.compartment + "]"
    if metabolite.id.endswith(trail):
        metabolite.id = metabolite.id.split(trail)[0] + "_" + metabolite.compartment



In [126]:
ecModel.summary()

### 4. Export model

In [127]:
#Save model as .xml & .json:
cobra.io.save_json_model(ecModel, "models/ecYeastGEM.json")